In [3]:
# -*- coding: utf-8 -*-
"""
IVDD binary training (ivdd vs normal) - train2 (tail_set-centered + per-file min-max)
変更点:
- プロジェクトルートを自動検出（scripts配下にdataが出来ない）
- 学習曲線: loss/acc を1枚のPNGで保存 (fig/curve_YYYYMMDD.png)
- 学習モデル: train2_model に YYYYMMDD ベース名で best/final を保存
- Val誤分類ウィンドウ一覧を train/val_misclassified/val_misclassified_YYYYMMDD.csv で保存
- 既存の正規化（tail_set原点 + 各次元min-max）はそのまま
"""

# ===== 安定運用: GPU無効/ログ控えめ/スレッド抑制（必要なら） =====
import os
os.environ.setdefault("CUDA_VISIBLE_DEVICES", "-1")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")

import re, glob, datetime
from pathlib import Path
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

print("TF:", tf.__version__)
tf.random.set_seed(42)
np.random.seed(42)

# ========= ルート自動検出（scripts直下で実行してもOK） =========
def detect_project_root() -> Path:
    env = os.environ.get("IVDD_PROJECT_ROOT")
    if env:
        p = Path(env).expanduser().resolve()
        if (p / "data").is_dir() or (p / "scripts").is_dir():
            return p
    try:
        here = Path(__file__).resolve()
        if here.parent.name == "scripts":
            cand = here.parent.parent
            if (cand / "data").is_dir() or (cand / "scripts").is_dir():
                return cand
        if (here.parent / "data").is_dir() or (here.parent / "scripts").is_dir():
            return here.parent
    except NameError:
        pass
    cwd = Path.cwd()
    for cand in [cwd] + list(cwd.parents):
        if (cand / "data").is_dir() and (cand / "scripts").is_dir():
            return cand
    return cwd.parent if cwd.name == "scripts" else cwd

PROJ_ROOT   = detect_project_root()
TRAIN_DIR   = PROJ_ROOT / "data" / "train"
TRAIN_CSV_DIR = TRAIN_DIR / "train_csv"
FIG_DIR     = TRAIN_DIR / "fig"
MODEL_DIR   = TRAIN_DIR / "train2_model"          # ← train2 のモデル置き場
VALERR_DIR  = TRAIN_DIR / "val_misclassified"     # ← 新設: 誤分類CSV置き場
for d in [FIG_DIR, MODEL_DIR, VALERR_DIR]:
    d.mkdir(parents=True, exist_ok=True)

CSV_GLOB = str(TRAIN_CSV_DIR / "*.csv")

# ========= パラメータ =========
KEYPOINTS = [
    "left back paw",
    "right back paw",
    "left front paw",
    "right front paw",
    "tail set",
]

USE_LIKELIHOOD      = False
MIN_KEEP_LIKELIHOOD = 0.6

SEQ_LEN  = 60
STRIDE   = 30
DIMS      = 10
N_HIDDEN  = 30

BATCH_SIZE = 30
EPOCHS     = 50
LR         = 1e-4
L2_LAMBDA  = 1e-4

VAL_SPLIT_BY_FILE = True

# バイナリ: 0=normal, 1=ivdd
CLASS_NAMES  = ["normal", "ivdd"]
CLASS_TO_IDX = {"normal": 0, "ivdd": 1}

# ランごとユニークID（日時）
DATE_STR = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")


# ========= ラベル推定 =========
def infer_label_from_filename(path: str) -> int:
    base = os.path.basename(path).lower()
    stem = os.path.splitext(base)[0]
    tokens = [t for t in re.split(r'[^a-z0-9]+', stem) if t]

    # 1) 先頭トークンが ivdd/normal + 任意の数字 を許容（例: ivdd1, normal20）
    if tokens:
        head = tokens[0]
        if re.fullmatch(r'ivdd\d*', head):
            return CLASS_TO_IDX["ivdd"]
        if re.fullmatch(r'normal\d*', head):
            return CLASS_TO_IDX["normal"]

    # 2) 他トークンに厳密一致（両方ヒットは避ける）
    if ("ivdd" in tokens) and ("normal" not in tokens):
        return CLASS_TO_IDX["ivdd"]
    if ("normal" in tokens) and ("ivdd" not in tokens):
        return CLASS_TO_IDX["normal"]

    # 3) 親ディレクトリ名に基づくフォールバック（サブフォルダ運用時）
    parent_tokens = [t for t in re.split(r'[^a-z0-9]+', os.path.dirname(path).lower()) if t]
    if ("ivdd" in parent_tokens) and ("normal" not in parent_tokens):
        return CLASS_TO_IDX["ivdd"]
    if ("normal" in parent_tokens) and ("ivdd" not in parent_tokens):
        return CLASS_TO_IDX["normal"]

    # 4) 最後の手段: どこかで ivdd/normal の直後が数字なら採用（IvddOct は除外）
    if re.search(r'(?<![a-z])ivdd(?=\d)', stem):
        return CLASS_TO_IDX["ivdd"]
    if re.search(r'(?<![a-z])normal(?=\d)', stem):
        return CLASS_TO_IDX["normal"]

    raise ValueError(f"ラベル不明: {base}（先頭を ivdd_ / normal_ または ivdd1_ / normal1_ にするか、上の判定を調整）")

# ========= DLC 読み取り & 正規化 =========
def _norm_name(s: str) -> str:
    return "".join(ch for ch in s.lower() if ch not in " _-")

def _resolve_keypoints(all_bodyparts, requested):
    norm2orig = {}
    for bp in all_bodyparts:
        k = _norm_name(bp)
        if k not in norm2orig:
            norm2orig[k] = bp
    resolved, missing = [], []
    for req in requested:
        k = _norm_name(req)
        if k in norm2orig:
            resolved.append(norm2orig[k])
        else:
            missing.append(req)
    if missing:
        raise ValueError(f"指定キーポイントがCSVに見つかりません: {missing}\n利用可能: {all_bodyparts}")
    return resolved

def read_dlc_5kp_xy(csv_path: str, keypoints, use_likelihood=True, min_keep_likelihood=0.6):
    df = pd.read_csv(csv_path, header=[0,1,2], index_col=0)
    bodyparts = list({bp for (_, bp, _) in df.columns})
    use_kps = _resolve_keypoints(bodyparts, keypoints)

    cols = {}
    for bp in use_kps:
        cols[f"{bp}_x"] = df.xs((bp, "x"), level=[1,2], axis=1)
        cols[f"{bp}_y"] = df.xs((bp, "y"), level=[1,2], axis=1)
    X_df = pd.concat(cols.values(), axis=1)
    X_df.columns = list(cols.keys())

    if use_likelihood:
        for bp in use_kps:
            try:
                lcol = df.xs((bp, "likelihood"), level=[1,2], axis=1).values.flatten()
                low = lcol < min_keep_likelihood
                for c in [f"{bp}_x", f"{bp}_y"]:
                    v = X_df[c].values
                    v[low] = np.nan
                    X_df[c] = v
            except KeyError:
                pass

    X_df = X_df.interpolate(method="linear", limit_direction="both", axis=0)
    X_df = X_df.bfill().ffill().fillna(0.0)
    X = X_df.values.astype(np.float32)  # (T,10)
    return X, use_kps

def normalize_tailset_minmax(X: np.ndarray, used_kps, ref_name: str = "tail set", eps: float = 1e-6) -> np.ndarray:
    low = [s.lower() for s in used_kps]
    if ref_name.lower() not in low:
        raise ValueError(f"'{ref_name}' が used_kps に見つかりません: {used_kps}")
    ref_idx = low.index(ref_name.lower())

    # 原点平行移動
    cx = X[:, 2*ref_idx]
    cy = X[:, 2*ref_idx + 1]
    Xc = X.copy()
    for i in range(len(used_kps)):
        Xc[:, 2*i]   -= cx
        Xc[:, 2*i+1] -= cy

    # 各次元 min-max（ファイル単位）
    x_min = Xc.min(axis=0, keepdims=True)
    x_max = Xc.max(axis=0, keepdims=True)
    Xn = (Xc - x_min) / (x_max - x_min + eps)
    return Xn

def make_windows(X: np.ndarray, seq_len: int, stride: int):
    n = X.shape[0]
    if n < seq_len:
        return np.empty((0, seq_len, X.shape[1]), dtype=X.dtype), []
    starts = list(range(0, n - seq_len + 1, stride))
    Xw = np.stack([X[s:s+seq_len] for s in starts], axis=0) if starts else np.empty((0, seq_len, X.shape[1]), dtype=X.dtype)
    return Xw, starts

def build_dataset(csv_paths, seq_len=SEQ_LEN, stride=STRIDE):
    """
    戻り:
      X: (N, T, D)
      y: (N,) 0/1
      file_ids: (N,) 元CSVファイル名
      starts: (N,) ウィンドウ開始フレーム
    """
    X_list, y_list, file_ids, starts_all = [], [], [], []
    used_kps_any = None

    for p in csv_paths:
        y_lab = infer_label_from_filename(p)
        X_raw, used_kps = read_dlc_5kp_xy(
            p, keypoints=KEYPOINTS, use_likelihood=USE_LIKELIHOOD, min_keep_likelihood=MIN_KEEP_LIKELIHOOD
        )
        if used_kps_any is None:
            used_kps_any = used_kps
        if X_raw.shape[1] != DIMS:
            raise ValueError(f"{Path(p).name}: 次元 {X_raw.shape[1]} != 期待 {DIMS}")

        # ★ tail_set中心 + min-max 正規化（維持）
        X_norm = normalize_tailset_minmax(X_raw, used_kps, ref_name="tail set")

        X_win, starts = make_windows(X_norm, seq_len, stride)
        if X_win.shape[0] == 0:
            print(f"[WARN] {Path(p).name}: フレーム不足（{seq_len}未満）でスキップ")
            continue

        X_list.append(X_win)
        y_list.append(np.full((X_win.shape[0],), y_lab, dtype=np.int64))
        file_ids += [Path(p).name] * X_win.shape[0]
        starts_all += starts

    if not X_list:
        raise RuntimeError("データが作れませんでした。CSV名に 'ivdd' / 'normal' を含めてください。")

    X = np.concatenate(X_list, axis=0)
    y = np.concatenate(y_list, axis=0)
    file_ids = np.array(file_ids)
    starts_all = np.array(starts_all, dtype=int)
    print(f"[INFO] 使用キーポイント実名: {used_kps_any}")
    return X, y, file_ids, starts_all

# ========= モデル =========
def build_lstm_model(seq_len: int, dims: int, n_hidden: int, l2_lambda: float = 1e-4) -> keras.Model:
    reg = keras.regularizers.l2(l2_lambda)
    inp = keras.Input(shape=(seq_len, dims), name="input")
    td  = keras.layers.TimeDistributed(
        keras.layers.Dense(n_hidden, activation="relu", kernel_regularizer=reg),
        name="td_dense"
    )(inp)
    x   = keras.layers.LSTM(n_hidden, return_sequences=True, kernel_regularizer=reg, name="lstm1")(td)
    x   = keras.layers.LSTM(n_hidden, kernel_regularizer=reg, name="lstm2")(x)
    out = keras.layers.Dense(1, kernel_regularizer=reg, name="logits")(x)  # from_logits=True
    return keras.Model(inp, out, name="ivdd_lstm")

# ========= データ読み込み =========
csv_files = sorted(glob.glob(CSV_GLOB))
if not csv_files:
    raise FileNotFoundError(f"CSV が見つかりません: {CSV_GLOB}")

X, y, file_ids, starts = build_dataset(csv_files, SEQ_LEN, STRIDE)
print("X:", X.shape, "y:", y.shape, "files:", len(np.unique(file_ids)))

# ========= 分割（ファイル単位推奨） =========
if VAL_SPLIT_BY_FILE:
    uniq = np.unique(file_ids)
    tr_files, va_files = train_test_split(uniq, test_size=0.2, random_state=42, shuffle=True)
    tr_mask = np.isin(file_ids, tr_files)
    va_mask = np.isin(file_ids, va_files)
    X_train, y_train = X[tr_mask], y[tr_mask]
    X_val,   y_val   = X[va_mask], y[va_mask]
    val_file_ids     = file_ids[va_mask]
    val_starts       = starts[va_mask]
else:
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    # ファイル名/開始は未知になるのでダミー
    val_file_ids = np.array(["unknown.csv"]*len(y_val))
    val_starts   = np.arange(len(y_val))

print("train:", X_train.shape, "val:", X_val.shape)

# ========= クラス不均衡対策 =========
unique_classes = np.unique(y_train)
if set(unique_classes) == {0, 1}:
    cls_w = compute_class_weight(class_weight="balanced", classes=np.array([0,1]), y=y_train)
    class_weight = {0: float(cls_w[0]), 1: float(cls_w[1])}
else:
    class_weight = None
print("class_weight:", class_weight)

# ========= 学習（EarlyStoppingなし） =========
model = build_lstm_model(SEQ_LEN, DIMS, N_HIDDEN, L2_LAMBDA)
model.summary()

opt = keras.optimizers.Adam(learning_rate=LR)
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
metrics = [keras.metrics.BinaryAccuracy(name="accuracy")]
model.compile(optimizer=opt, loss=loss_fn, metrics=metrics)

best_model_path  = MODEL_DIR / f"ivdd_lstm_{DATE_STR}_best.keras"
final_model_path = MODEL_DIR / f"ivdd_lstm_{DATE_STR}_final.keras"

callbacks = [
    keras.callbacks.ModelCheckpoint(
        str(best_model_path), monitor="val_accuracy",
        save_best_only=True, save_weights_only=False, mode="max", verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", mode="min",
        factor=0.5, patience=5, min_lr=1e-5, verbose=1
    ),
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    class_weight=class_weight,
    verbose=1,
    callbacks=callbacks
)

# ========= 学習曲線（loss/acc を1枚で保存） =========
curve_png = FIG_DIR / f"curve_{DATE_STR}.png"
plt.figure(figsize=(10,4))
plt.subplot(1,2,1); plt.plot(history.history["loss"], label="train"); plt.plot(history.history["val_loss"], label="val")
plt.title("Loss"); plt.legend()
plt.subplot(1,2,2); plt.plot(history.history["accuracy"], label="train"); plt.plot(history.history["val_accuracy"], label="val")
plt.title("Accuracy"); plt.legend()
plt.tight_layout(); plt.savefig(curve_png, dpi=150); plt.close()
print(f"[INFO] saved curve: {curve_png}")

# ========= Val評価 & 誤分類CSV =========
logits_val = model.predict(X_val, batch_size=64)
p_ivdd = tf.math.sigmoid(logits_val).numpy().ravel()   # ivdd=1 の確率
y_pred = (p_ivdd >= 0.5).astype(int)

print("\n[Window-level] classification_report:")
print(classification_report(y_val, y_pred, target_names=CLASS_NAMES, digits=4))
cm = confusion_matrix(y_val, y_pred)
print("[Window-level] confusion matrix:\n", cm)

# 誤分類ウィンドウを書き出し（要件: train配下の新フォルダにYYYYMMDD名）
val_df = pd.DataFrame({
    "file": val_file_ids,
    "win_start": val_starts,           # ウィンドウ開始フレーム
    "true": y_val,
    "pred": y_pred,
    "p_ivdd": p_ivdd,
    "p_normal": 1.0 - p_ivdd,
})
errors_df = val_df[val_df["true"] != val_df["pred"]].copy()
err_csv = VALERR_DIR / f"val_misclassified_{DATE_STR}.csv"
errors_df.to_csv(err_csv, index=False, encoding="utf-8-sig")
print(f"[INFO] saved misclassified windows: {err_csv}  (rows={len(errors_df)})")

# ========= 最終モデル保存 =========
model.save(str(final_model_path))
print(f"[INFO] saved final model to: {final_model_path}")

print("[DONE] training complete.")


TF: 2.19.0
[INFO] 使用キーポイント実名: ['left back paw', 'right back paw', 'left front paw', 'right front paw', 'tail set']
X: (1460, 60, 10) y: (1460,) files: 198
train: (1166, 60, 10) val: (294, 60, 10)
class_weight: {0: 1.1, 1: 0.9166666666666666}


Model: "ivdd_lstm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 60, 10)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ td_dense (TimeDistributed)      │ (None, 60, 30)         │           330 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm1 (LSTM)                    │ (None, 60, 30)         │         7,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm2 (LSTM)                    │ (None, 30)             │         7,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ logits (Dense)                  │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,001 (58.60 KB)

 Trainable params: 15,001 (58.60 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4618 - loss: 0.7121
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to c:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\train\train2_model\ivdd_lstm_20251211-045755_best.keras
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.4616 - loss: 0.7119 - val_accuracy: 0.5000 - val_loss: 0.7052 - learning_rate: 1.0000e-04
Epoch 2/50
37/39 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4622 - loss: 0.7061
Epoch 2: val_accuracy did not improve from 0.50000
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4616 - loss: 0.7059 - val_accuracy: 0.5000 - val_loss: 0.7034 - learning_rate: 1.0000e-04
Epoch 3/50
36/39 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4625 - loss: 0.7043
Epoch 3: val_accuracy did not improve from 0.50000
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.4616 - loss: 0.7042 - val_accur